In [2]:
import numpy as np
import pandas as pd
import edhec_risk_kit as erk

%load_ext autoreload
%autoreload 2

# Quiz 4

B1 is a 15 Year Bond with a Face Value of $1000 that pays a 5% coupon semi-annually (2 times a year)

B2 is a 5 Year Bond with a Face value of $1000 that pays a 6% coupon quarterly (4 times a year)

B3 is a 10 Year Zero-Coupon Bond with a Face Value of $1000

(Hint: you can still use the erk.bond_cash_flows() and erk.bond_price() by setting the coupon amount to 0% and coupons_per_year to 1)  Assume the yield curve is flat at 5%. Duration refers to Macaulay Duration

Hint: the macaulay_duration function gives as output the duration expressed in periods and not in years. If you want to get the yearly duration you need to divide the duration for coupons_per_year; 

e.g.: duarion_B2 = erk.macaulay_duration(flows_B2, 0.05/4)/4

In [12]:
flow_b1 = erk.bond_cash_flows(15, 1000, .05, 2)

In [13]:
flow_b2 = erk.bond_cash_flows(5, 1000, .06, 4)

In [14]:
flow_b3 = erk.bond_cash_flows(10, 1000, 0, 1)

In [15]:
erk.bond_price(15, 1000, .05, 2, .05)

0    1000.0
dtype: float64

In [16]:
erk.bond_price(5, 1000, .06, 4, .05)

0    1043.99829
dtype: float64

In [17]:
erk.bond_price(10, 1000, 0, 1, .05)

0    613.913254
dtype: float64

In [34]:
flow_b1

1       25.0
2       25.0
3       25.0
4       25.0
5       25.0
6       25.0
7       25.0
8       25.0
9       25.0
10      25.0
11      25.0
12      25.0
13      25.0
14      25.0
15      25.0
16      25.0
17      25.0
18      25.0
19      25.0
20      25.0
21      25.0
22      25.0
23      25.0
24      25.0
25      25.0
26      25.0
27      25.0
28      25.0
29      25.0
30    1025.0
dtype: float64

In [21]:
flow_b1.index.shape

(30,)

In [43]:
discounted_flow = erk.discount(flow_b1.index, 0.05/2)[0]*flow_b1

In [44]:
weights = discounted_flow/discounted_flow.sum()

In [45]:
np.average(flow_b1.index, weights=weights)/2
# B1 듀레이션

10.72677495379012

In [40]:
def macaulay_duration(flows, discount_rate):
    """
    현금흐름으로 듀레이션 계산
    """
    discounted_flows = erk.discount(flows.index, discount_rate)[0]*flows
    weights = discounted_flows/discounted_flows.sum()
    return np.average(flows.index, weights=weights)

In [46]:
macaulay_duration(flow_b1, .05/2)/2

10.72677495379012

In [47]:
macaulay_duration(flow_b2, .05/4)/4

4.373363222636413

In [48]:
macaulay_duration(flow_b3, .05/1)

10.0

Assume a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away,
 respectively. What is the Duration of the liabilities?

In [51]:
liabilities = pd.Series(index=[3, 5, 10], data=[100000, 200000, 300000])
liabilities

3     100000
5     200000
10    300000
dtype: int64

In [52]:
macaulay_duration(liabilities, .05)

6.750917852744651

Assuming the same set of liabilities as the previous question (i.e. a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away,
 respectively) build a Duration Matched Portfolio of B1 and B2 to match these liabilities. What is the weight of B2 in the portfolio? 
 
 (Hint: the code
 we developed in class erk.match_durations() assumes that all the bonds have the same number of coupons
 per year. This is not the case here, so you will either need to enhance the code or compute the weight directly

In [64]:
def match_durations(cf_t, cf_s, cf_l, discount_rate):
    """
    자산 부채 듀레이션 맞추기: d_s의 비율 return
    """
    d_t = macaulay_duration(cf_t, discount_rate)
    d_s = macaulay_duration(cf_s, discount_rate)
    d_l = macaulay_duration(cf_l, discount_rate)
    return (d_l - d_t)/(d_l - d_s)

In [57]:
d_l = macaulay_duration(flow_b1, .05/2)/2

In [58]:
d_s = macaulay_duration(flow_b2, .05/4)/4

In [59]:
d_t = macaulay_duration(liabilities, .05)

In [61]:
(d_l - d_t)/(d_l - d_s)

0.6257830075060314

Assuming the same liabilities as the previous questions (i.e. a sequence of 3 liabilities of $100,000, $200,000 and $300,000 that are 3, 5 and 10 years away,
 respectively), build a Duration Matched Portfolio of B2 and B3 to match the liabilities.
What is the weight of B2 in this portfolio?

In [62]:
d_l = macaulay_duration(flow_b3, .05)

In [63]:
(d_l - d_t)/(d_l - d_s)

0.5774465770256697